In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os
import sys
import pdfminer.high_level # ! pip install pdfminer.six

In [3]:
from tqdm import tqdm
import time
from string import punctuation
import nltk

In [40]:
tqdm.pandas(desc="apply progress bar")

#### Собираем таблицы после парсинга

In [4]:
med_df = pd.read_csv('med_df.csv')
med_df_contraf = pd.read_csv('med_df_contraf.csv')
med_df_doubt = pd.read_csv('med_df_doubt.csv')
med_df_falsh = pd.read_csv('med_df_falsh.csv')
med_df_original = pd.read_csv('med_df_original.csv')
med_df_unger = pd.read_csv('med_df_unger.csv')

In [5]:
med_df = pd.concat([med_df, med_df_contraf, med_df_doubt, med_df_falsh, med_df_original, med_df_unger]).reset_index()

In [6]:
med_df.drop(['index', 'Unnamed: 0'], axis=1, inplace=True)
med_df.drop_duplicates(inplace=True)
med_df.reset_index(inplace=True)
med_df.drop(['index'], axis=1, inplace=True)


In [7]:
med_df.shape

(24979, 9)

In [8]:
med_given = pd.read_excel('./Report_Database.xlsx', encoding="utf_8")

In [195]:
dict_reasons_init = {
'Упаковка': 1,
'инструкция на турецком языке': 1,

'Маркировка и инструкция': 2,
'Маркировка на ампулах стерта': 2,
'Маркировка': 2,
    
'Отзыв декларации о соответствии': 5,
'Отзыв производителем': 5,
'Отмена государственной регистрации': 5,
'Прекращение действия декларации о соответствии': 5,
'Отзыв производителем декларации о соответствии': 5,
'В связи с преостановлением реализации фармацевтической субстанции': 5,
'Необходимость гармонизации регистрационного досье и нормативной документации': 5,
'Необходимость гармонизации регистрационных документов., ': 5,
'Отмена действия сертификата соответствия': 5,
    
'Описание (': 6,
'Описание,': 6,
'Цветность': 6,
'запись о которой отсутствует в Государственном реестре лекарственных средств': 6,

'Количественное определение': 8,
'Количественное определение:Пропилпарагидроксибензоат': 8,
'Герметичность, ': 8,
'Растворение': 8,
    
'Микробиологическая чистота': 10,

'Фальсификация' : 10, 
'Подлинность' : 10, 
'Признаки фальсификации' : 10,
'Отмена действия сертификата соответствия' : 10,
'На этикетках флаконов отсутствуют серия и срок годности' : 10,
'Указание на первичной и вторичной упаковках: Дата изг Годен до' : 10, 
'Стабильность препарата' : 10,
'Развитие нежелательных реакций' : 10,
'Растворение' : 10, 
'Несоответствие в процессе производства' : 10,
'Выявление разночтений в документах': 10,
'Выявлены несоответствия утвержденным нормам спецификации, в ходе изучения стабильности': 10,
'Механические включения': 10,
'Обнаружение частиц полиуретана в упаковочном оборудовании': 10,
'Остаточные органические растворители': 10,

'Показатель в письме не указан': 1,
'Посторонние примеси': 10,
'Развитие нежелательных реакций': 10
 }

In [196]:
dict_reasons = {
'Упаковка': 1,
'инструкция турецком языке': 1,

'Маркировка инструкция': 2,
'Маркировка ампулах стерта': 2,
'Маркировка': 2,
    
'Отзыв декларации соответствии': 5,
'Отзыв производителем': 5,
'Отмена государственной регистрации': 5,
'Прекращение действия декларации соответствии': 5,
'Отзыв производителем декларации соответствии': 5,
'связи преостановлением реализации фармацевтической субстанции': 5,
'Необходимость гармонизации регистрационного досье нормативной документации': 5,
'Необходимость гармонизации регистрационных документов': 5,
'Отмена действия сертификата соответствия': 5,
    
'Описание (': 6,
'Описание,': 6,
'Цветность': 6,
'запись отсутствует Государственном реестре': 6,

'Количественное определение': 8,
'Герметичность': 8,
'Растворение': 8,
    
'Микробиологическая чистота': 10,

'Фальсификация' : 10, 
'Подлинность' : 10, 
'Признаки фальсификации' : 10,
'Отмена действия сертификата соответствия' : 10,
'этикетках флаконов отсутствуют серия срок годности' : 10,
'Указание первичной вторичной упаковках' : 10, 
'Стабильность препарата' : 10,
'Развитие нежелательных реакций' : 10,
'Растворение' : 10, 
'Несоответствие процессе производства' : 10,
'Выявление разночтений документах': 10,
'Выявлены несоответствия утвержденным нормам спецификации изучения стабильности': 10,
'Механические включения': 10,
'Обнаружение частиц полиуретана упаковочном оборудовании': 10,
'Остаточные органические растворители': 10,

'Показатель письме указан': 1,
'Посторонние примеси': 10,
'Развитие нежелательных реакций': 10
 }

In [165]:
def my_tokenizer(text):
    out = []
    for i in nltk.word_tokenize(str(text)):
        word = ''
        for m in i.lower():
            if m not in punctuation:
                word += m
        if len(word)>1:
            out.append(word)
    return(set(out))


In [197]:
reasons_a, reasons_b = [], []
for a, b in dict_reasons.items():
    reasons_a.append(a)
    reasons_b.append(b)
reasons = pd.DataFrame(reasons_a, reasons_b)

In [198]:
reasons_a, reasons_b = [], []
for a, b in dict_reasons_init.items():
    reasons_a.append(a)
    reasons_b.append(b)
reasons_pure = pd.DataFrame(reasons_a, reasons_b)

In [199]:
reasons.reset_index(inplace=True)
reasons_list = list(reasons.sort_values(by='index', ascending=False)[0])
reasons_parts = list(map(my_tokenizer, reasons_list))

In [200]:
reasons_pure.reset_index(inplace=True)
reasons_list_pure = list(reasons_pure.sort_values(by='index', ascending=False)[0])

In [201]:
alpha = 1
def get_reason(text):
    if text == '':
        return set(), ''
    max_res = 0
    max_set_i = 0
    best_inter = []
    for set_i in range(len(reasons_parts)):
        if reasons_list_pure[set_i] in text:
            return set(), reasons_list_pure[set_i]
        if len(reasons_parts[set_i]) > 0:
            inter = my_tokenizer(text).intersection(reasons_parts[set_i])
            #current_res = alpha * len(inter) / len(reasons_parts[set_i]) + (1 - alpha) * (set_i / 10)
            current_res = alpha * len(inter) + (1 - alpha) * set_i
            if current_res > max_res:
                max_set_i = set_i
                max_res = current_res
                best_inter = inter
    if max_res > (1 - alpha) * set_i:           
        return best_inter, reasons_list[max_set_i]
    return set(), ''

#### Преобразуем pdf в txt

In [24]:
med_df['Приказ'] = ''
med_df['Причины забраковки'] = ''

In [25]:
for i in tqdm(range(len(med_df))):
    url = 'http://www.roszdravnadzor.ru/services/lssearch' + str(med_df.loc[i, 'Информационное письмо'])
    succsess = False
    if url != 'http://www.roszdravnadzor.ru/services/lssearch':
        try:
            urllib.request.urlretrieve(url, os.path.join('../AI.Hack/', "file.pdf"))
            with open('out.txt', 'w') as f:
                with open('file.pdf', 'rb') as file:
                    try:
                        pdfminer.high_level.extract_text_to_fp(file, f)
                        succsess = True
                    except:
                        pass
        except:
            pass
        if succsess:
            with open('out.txt', 'r', encoding='utf-8') as file2:
                text = '. '.join(file2.readlines())
                if len(text)>10:
                    med_df.loc[i, 'Приказ'] = text
#                 print(get_reason(text))
            

100%|██████████| 24979/24979 [4:31:32<00:00,  1.53it/s]  


In [31]:
med_df.to_csv('med_df_text2.csv', encoding='utf-8')

In [30]:
len(med_df['Приказ'].unique())

3347

In [38]:
sum(med_df['Приказ'] != '')/len(med_df['Приказ'])

0.36174386484647103

#### Собираем таблицу с причинами

In [187]:
med_unique = med_df[['Приказ', 'Информационное письмо']][med_df['Приказ']!=''].drop_duplicates()
med_unique.reset_index(inplace=True)
med_unique.drop('index', axis=1, inplace=True)

In [188]:
med_unique['Причина']=''
med_unique['Причина_части']=[set() for i in range(len(med_unique))]

In [203]:
for i in tqdm(range(len(med_unique))):
    a, b = get_reason(str(med_unique.loc[i, 'Приказ']))
    med_unique.loc[i, 'Причина'] = b
    med_unique.loc[i, 'Причина_части']= a

100%|██████████| 3346/3346 [24:30<00:00,  2.28it/s] 


In [207]:
med_unique.to_csv('med_unique_docs.csv', encoding='utf-8')

In [208]:
med_unique

,Приказ,Информационное письмо,Причина,Причина_части
0,219957МинистерствоздравоохраненияРоссийскойФед...,?downloadletter=21256,Несоответствие процессе производства,{производства}
1,Министерство здравоохранения Российской Федера...,?downloadletter=23684,Маркировка,{}
2,Министерство здравоохранения Российской Федера...,?downloadletter=23655,Маркировка,{}
3,Министерство здравоохранения Российской Федера...,?downloadletter=24685,Прекращение действия декларации соответствии,"{декларации, соответствии}"
4,Министерство здравоохранения Российской Федера...,?downloadletter=27619,Количественное определение,{}
5,п Министерство здравоохранения Российской ...,?downloadletter=25568,Количественное определение,"{количественное, определение}"
6,п 4 s Министерство здравоохранения Российск...,?downloadletter=26945,Стабильность препарата,{препарата}
7,Министерство здравоохранения Российской Федера...,?downloadletter=27507,Стабильность препарата,{препарата}
8,1Министерство здравоохраневвя Российской Федер...,?downloadletter=22933,этикетках флаконов отсутствуют серия срок годн...,"{срок, флаконов}"
9,"109074,Москва,Славянскаяпл.,д.4,стр.1тел.:6984...",?downloadletter=15342,Маркировка,{}


In [147]:
data_common = pd.merge(med_df.drop('Причины забраковки', axis=1), med_given, how='left', on='Серия')
data_common = data_common[~pd.isnull(data_common['Лаборатория'])]
data_common.reset_index(inplace=True)

In [202]:
#тестирование
for i in range(0,70):#len(med_df)):
    url = 'http://www.roszdravnadzor.ru/services/lssearch' + data_common.loc[i, 'Информационное письмо']
    if url != 'http://www.roszdravnadzor.ru/services/lssearch':
#         print(url)
        urllib.request.urlretrieve(url, os.path.join('../AI.Hack/', "file.pdf"))
        with open('out%d.txt'%i, 'w') as f:
            with open('file.pdf', 'rb') as file:
                try:
                    pdfminer.high_level.extract_text_to_fp(file, f)
                except:
                    print('not pdf')
                
#         f.close()
        with open('out%d.txt'%i, 'r', encoding='utf-8') as file2:
            text = ''.join(file2.readlines())
#             print(len(text))
            print('pred:', get_reason(text))
            print('true:', data_common.loc[i, 'Причины забраковки'])
            print('')

pred: (set(), 'Маркировка')
true: Количественное определение, 

pred: (set(), 'Маркировка')
true: Количественное определение, 

pred: (set(), 'Количественное определение')
true: Отзыв декларации о соответствии РОСС SI.ФМ08.Д45272 от 24.04.2014, Отзыв декларации о соответствии РОСС SI.ФМ08.Д87003 от 09.06.2014, 

not pdf
pred: (set(), '')
true: Отзыв декларации о соответствии РОСС SI.ФМ08.Д45272 от 24.04.2014, Отзыв декларации о соответствии РОСС SI.ФМ08.Д87003 от 09.06.2014, 

pred: (set(), 'Количественное определение')
true: Количественное определение.Лимонная кислота., Прекращение действия декларации о соответствии, 

not pdf
pred: (set(), '')
true: Количественное определение.Лимонная кислота., Прекращение действия декларации о соответствии, 

pred: (set(), '')
true: Маркировка, 

pred: (set(), '')
true: Маркировка, 

pred: (set(), 'Количественное определение')
true: Количественное определение (амилметакрезол, 2,4-дихлорбензиловый спирт), Отзыв декларации о соответствии, 

pred: (set

KeyboardInterrupt: 

#### Способ 2

Находится в файле AI.Hack_clastering